# Super Simple Keras model on TriFeature for multiplicity determination

In [1]:
features = ["T", "E", "Size", "EToF", "EnergyMoment", "TSpawn", "MaxEHit", "X", "Y", "Z"]
label = "prim"
dp = 30
neutrons = [1, 2, 3, 4]

In [2]:
import sys
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import joblib
import sklearn.preprocessing
from sklearn.metrics import (
    balanced_accuracy_score,
    confusion_matrix,
    plot_confusion_matrix,
)

sys.path.append("..")
from helpers import filename_for

Welcome to JupyROOT 6.16/00


In [3]:
files = [filename_for(15, dp, 600, 500, n, "inclxx", s, "clusterfeature.parquet") for n in neutrons for s in range(20)]
dfs = [pd.read_parquet(file) for file in files]
data = pd.concat(dfs, ignore_index=True).sample(frac=1, random_state=1337)

data = data[(abs(data["X"]) < 125) & (abs(data["Y"]) < 125) & (data["T"] < 100)]

print(data["prim"].value_counts())
display(data)

0.0    8499721
1.0    1779876
Name: prim, dtype: int64


,i_event,prim,T,E,Size,EToF,EnergyMoment,TSpawn,MaxEHit,X,Y,Z
3499177,1731.0,1.0,63.755749,149.914871,6.0,607.207764,6.798460e+00,1.218181,64.293854,-0.682722,-22.500000,1522.5
8869071,3484.0,0.0,74.362877,15.335159,4.0,580.952515,3.743494e+00,0.394498,9.338829,37.500000,-1.970521,1757.5
2585062,5713.0,0.0,68.605553,37.585819,1.0,600.481689,2.278178e-13,0.000000,37.585819,-76.964607,2.500000,1632.5
2894202,5004.0,0.0,74.553833,34.140911,2.0,625.573181,2.255625e+00,0.020336,23.147533,-6.335686,17.500000,1792.5
3878350,5814.0,0.0,86.484261,7.983827,1.0,264.821777,0.000000e+00,0.000000,7.983827,98.367348,67.500000,1622.5
...,...,...,...,...,...,...,...,...,...,...,...,...
480729,1884.0,0.0,79.410904,2.277636,1.0,296.172211,0.000000e+00,0.000000,2.277636,-72.500000,-76.763123,1547.5
3361959,3033.0,1.0,63.699387,613.164795,38.0,609.370972,3.672803e+01,5.340049,52.407722,3.392983,12.500000,1522.5
7087336,7733.0,0.0,70.274147,246.902069,11.0,568.768555,1.244804e+01,2.275502,52.906334,6.802186,37.500000,1652.5
8315069,6834.0,0.0,78.157318,2.550865,1.0,425.412720,0.000000e+00,0.000000,2.550865,41.877541,82.500000,1702.5


In [4]:
#scaler = sklearn.preprocessing.RobustScaler()
#data[features] = scaler.fit_transform(data[features])
#joblib.dump(scaler, "models/keras-scaler.pkl.gz")
#display(data)

In [5]:
prim1 = data[data["prim"] == 1]
prim0 = data[data["prim"] == 0].sample(n=len(prim1.index), random_state=1337)
balanced_data = pd.concat([prim0, prim1], ignore_index=True).sample(frac=1, random_state=1337)

print(balanced_data["prim"].value_counts())

1.0    1779876
0.0    1779876
Name: prim, dtype: int64


In [6]:
msk = np.random.rand(len(balanced_data)) < 0.8
traindata = balanced_data[msk]
testdata = balanced_data[~msk]

print(traindata.shape, testdata.shape)

(2848063, 12) (711689, 12)


In [7]:
x_train = traindata[features].values
y_train = traindata[[label]].values.ravel()

x_val = testdata[features].values
y_val = testdata[[label]].values.ravel()

In [8]:
#from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.classification import AutoSklearnClassifier

cls = AutoSklearnClassifier(n_jobs=10, memory_limit=50000, time_left_for_this_task=36000)
cls.fit(x_train, y_train)

/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


AutoSklearnClassifier(memory_limit=50000, n_jobs=10, per_run_time_limit=36000,
                      time_left_for_this_task=36000)

[WARNING] [2021-03-11 15:24:42,493:Client-AutoML(1):7d09d9ec-8275-11eb-bcb8-1866da859056] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (35996.181343)
[WARNING] [2021-03-11 15:24:42,493:Client-AutoML(1):7d09d9ec-8275-11eb-bcb8-1866da859056] Capping the per_run_time_limit to 17998.0 to have time for a least 2 models in each process.
[ERROR] [2021-03-11 15:24:42,933:Client-AutoMLSMBO(1)::7d09d9ec-8275-11eb-bcb8-1866da859056] Error getting metafeatures: Can't pickle local object 'Client.__init__.<locals>.<lambda>'


/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)
/home/jmayer/.pyenv/versions/3.7.6/lib/python3.7/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at

In [9]:
print(cls.sprint_statistics())

auto-sklearn results:
  Dataset name: 7d09d9ec-8275-11eb-bcb8-1866da859056
  Metric: accuracy
  Best validation score: 0.913777
  Number of target algorithm runs: 54
  Number of successful target algorithm runs: 36
  Number of crashed target algorithm runs: 0
  Number of target algorithms that exceeded the time limit: 17
  Number of target algorithms that exceeded the memory limit: 1



In [10]:
x_test = testdata[features].values
y_true = testdata[[label]].values.ravel()

y_pred = cls.predict(x_test)
bac = balanced_accuracy_score(y_true, y_pred)

print(bac)

0.913545781954203


In [11]:
#joblib.dump(cls, "models/autosklearn.pkl")

['models/autosklearn.pkl']